# Pseudobulking

In [1]:
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse
import scipy.io as sio

In [2]:
adata = sc.read("../scanpy/mm_nk.h5ad")
bioprojects = pd.read_csv("samples_to_project/mm_samples_to_project.csv", 
                          squeeze=True, 
                          index_col="Name").to_dict()
samples = adata.obs.batch
samples = samples.replace(bioprojects)
adata.obs["bioprojects"] = samples
adata = adata[adata.obs.new_label != "Others"]

In [3]:
adata.write("mm_nk.h5ad")

/home/jacopo/miniconda3/envs/scanpy_env/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c


In [4]:
pseudocounts = pd.DataFrame()
exp_design = pd.DataFrame(columns=["Sample", "Condition", "Batch"])
pseudocounts["Symbol"] = adata.var_names.to_list()
pseudocounts = pseudocounts.set_index("Symbol")
cells = adata.obs.new_label.values.to_list()
cells = list(set(cells))
cells.sort()
for cell in cells:
    adata_cell = adata[adata.obs["new_label"].isin([cell])]
    batches = adata_cell.obs.batch.values.to_list()
    batches = list(set(batches))
    batches.sort()
    for batch in batches:
        adata_tmp = adata_cell[adata_cell.obs["batch"].isin([batch])]
        adata_tmp
        mtx = adata_tmp.X.sum(0).tolist()[0]
        mtx=[int(x) for x in mtx]
        name = cell + "_" + batch
        project = list(set(adata_tmp.obs.bioprojects.values))[0]
        pseudocounts[name] = mtx
        row = [name, cell, project]
        exp_design.loc[len(exp_design)] = row

In [5]:
pseudocounts

,NK exhausted_MGUS_CD138nCD45p_2,NK exhausted_MGUS_CD138nCD45p_3,NK exhausted_MGUS_CD138nCD45p_4,NK exhausted_MGUS_CD138nCD45p_5,NK exhausted_MGUS_CD138n_1,NK exhausted_PMM_BM_1,NK exhausted_PMM_BM_10,NK exhausted_PMM_BM_11,NK exhausted_PMM_BM_2,NK exhausted_PMM_BM_3,...,NK resident_SMM_CD138nCD45p_8,NK resident_SMM_CD138nCD45p_9,NK resident_SMM_CD138n_3,NK resident_SMM_CD138n_4,NK resident_SMM_CD138n_5,NK resident_SMM_CD138n_6,NK resident_post-transplant_BM_1,NK resident_pre-transplant_BM_1,NK resident_pre-transplant_BM_2,NK resident_remission_BM_1
Symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,0,3,3,1,2,2,0,0,4,0,...,0,0,0,2,1,0,1,0,9,0
A1BG-AS1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,0,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
A2M-AS1,0,2,2,3,2,2,2,0,0,1,...,0,0,1,2,1,0,0,0,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bP-21264C1.2,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bP-2171C21.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bP-2189O9.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
pseudocounts.columns = pseudocounts.columns.str.replace(' ', '_')
pseudocounts.columns = pseudocounts.columns.str.replace('-', '_')
pseudocounts.to_csv("Pseudobulk_mm.csv")
exp_design = exp_design.set_index("Sample")
exp_design.index = exp_design.index.str.replace(' ', '_')
exp_design.index = exp_design.index.str.replace('-', '_')
exp_design["Condition"] = exp_design["Condition"].str.replace(' ', '_')
exp_design["Condition"] = exp_design["Condition"].str.replace('-', '_')
exp_design.to_csv("Experiment_design_mm.csv")

In [7]:
exp_design.index

Index(['NK_exhausted_MGUS_CD138nCD45p_2', 'NK_exhausted_MGUS_CD138nCD45p_3',
       'NK_exhausted_MGUS_CD138nCD45p_4', 'NK_exhausted_MGUS_CD138nCD45p_5',
       'NK_exhausted_MGUS_CD138n_1', 'NK_exhausted_PMM_BM_1',
       'NK_exhausted_PMM_BM_10', 'NK_exhausted_PMM_BM_11',
       'NK_exhausted_PMM_BM_2', 'NK_exhausted_PMM_BM_3',
       ...
       'NK_resident_SMM_CD138nCD45p_8', 'NK_resident_SMM_CD138nCD45p_9',
       'NK_resident_SMM_CD138n_3', 'NK_resident_SMM_CD138n_4',
       'NK_resident_SMM_CD138n_5', 'NK_resident_SMM_CD138n_6',
       'NK_resident_post_transplant_BM_1', 'NK_resident_pre_transplant_BM_1',
       'NK_resident_pre_transplant_BM_2', 'NK_resident_remission_BM_1'],
      dtype='object', name='Sample', length=144)

In [8]:
pseudocounts_clean = pseudocounts
pseudocounts.shape

(48361, 144)

In [9]:
# Remove the genes expressed by less than the 10% of samples
# The samples are the sum of counts of all the cells in the specific sample, if the counts are 0 in more then the 10% of samples
# it suggest that the gene is expressed in a minimal fraction of cells
ten_perch_samples = pseudocounts.shape[1]/10
pseudocounts_clean = pseudocounts_clean[pseudocounts_clean.ne(0).sum(1).gt(ten_perch_samples)] 
pseudocounts_clean.shape

(14136, 144)

In [10]:
# Remove the genes whose expression is less than 10 in all the samples
# Similarly to the previous ste
# The samples are the sum of counts of all the cells in the specific sample, if the counts are less than 10
# it suggest that the gene is expressed in a minimal fraction of cells
pseudocounts_clean = pseudocounts_clean[pseudocounts_clean.max(1).gt(10)] 
pseudocounts_clean.shape

(12421, 144)

In [11]:
pseudocounts_clean.to_csv("Pseudobulk_clean_mm.csv")